In [1]:
using Pkg
using Distributions
using ForwardDiff
using LinearAlgebra
using Optim
using Plots
using Random
using SSM

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [12]:
Random.seed!(1234)
g = 9.81
l=1
# time differential
dt = 0.01
# transition matrix
A = Matrix{Float64}([1.0 dt; -g/l*dt 1.0])
# Initial state
x0 = [0.0, 1.0]
# Define the LDS model parameters
C = Matrix{Float64}(I(2))  # Observation matrix (assuming direct observation)
Q = Matrix{Float64}(0.01 * I(2))  # Process noise covariance
observation_noise_std = 0.5
R = Matrix{Float64}((observation_noise_std^2) * I(2))  # Observation noise covariance
P0 = Matrix{Float64}(0.1*I(2))  # Initial state covariance

lds = SSM.GaussianLDS(;A=A, C=C, Q=Q, R=R, x0=x0, P0=P0, obs_dim=2, latent_dim=2, fit_bool=[true, true, true, true, true, true])

x, y = SSM.sample(lds, 100)

([-0.11375626857646887 1.343805512755226; -0.05829349566953871 1.2863979124749656; … ; 0.38927459888641114 0.060924881414345565; 0.2173556424880684 0.0922845673904967], [-0.32355107704589325 1.7032604814881955; 0.9690880323624795 1.4488442771984236; … ; -0.039807549312631696 0.06407608359556304; 0.20445211038157346 -0.0018833125233974485])

In [13]:
x, p_smooth, inverse_offdiag = SSM.smooth(lds, y)

([-0.12707975276822364 1.3113207579443449; -0.11729631582924958 1.339275066249993; … ; 0.3465653283138288 -0.25812781058954004; 0.3386174371376887 -0.28096269403659857], [0.02997425975735998 0.004336333100060945; 0.02778713623739736 0.0029746844624128493; … ; 0.03736433760488541 -0.00494511761303281; 0.0440731042944332 -0.007583462230285044;;; 0.004336333100060943 0.03230247103856569; 0.002974684462412851 0.02988592579621252; … ; -0.004945117613032819 0.040343522341738; -0.0075834622302850575 0.04814472469529661], [0.0 0.0; 0.02427920404227143 0.0020603290423333565; … ; 0.03035141007521449 -0.005801219493208781; 0.03587969848918758 -0.008279383780838527;;; 0.0 0.0; 0.00472901824481268 0.02639872610250525; … ; -0.0024822724610134507 0.03295674904757299; -0.004367002297707153 0.039258306134208186])

In [14]:
fit!(lds, y, max_iter=1)

1-element Vector{Float64}:
 395.2873662986955

In [24]:
lds.state_model.Q

2×2 Matrix{Float64}:
 0.0096097    0.000313099
 0.000313099  0.0101331

In [17]:
# test out the LDS model
lds_old = SSM.LDS(;A=A, H=C, Q=Q, R=R, x0=x0, p0=P0, obs_dim=2, latent_dim=2)

KalmanFilterEM!(lds_old, y, 1, 1e-6, RTSSmoothing())

1-element Vector{Any}:
 -173.66955056880425

In [25]:
lds_old.Q

2×2 Matrix{Float64}:
 0.0096097    0.000313099
 0.000313099  0.0101331

In [ ]:
x, p_smooth, inverse_offdiag = SSM.DirectSmoother(lds_old, y)